# Data cleaning and formatting

In [5]:
import os
import re
import csv
import unicodedata
from typing import Tuple
import pandas as pd

# Paths
INPUT_CSV   = "/content/gbv_2023Q4.csv"
REPAIRED_CSV = "/content/dataset_gbv_repaired.csv"
CLEAN_CSV    = "/content/dataset_gbv_clean.csv"

# Allowed values
ALLOWED_SENTIMENT = {"POS", "NEG", "-"}
ALLOWED_INFO      = {"INFO", "NOINFO", "-"}

# Allowed emotions
ALLOWED_EMOTIONS = {
    "GIOIA","TRISTEZZA","RABBIA","PAURA","DISGUSTO","SORPRESA","FIDUCIA","ATTESA","NEUTRA"
}

# Minimal patterns and sets
ISO_Z_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$")
INFO_SET = ALLOWED_INFO

# Row repair: recompose TEXT and realign last 4 columns
# Output columns: [ID, DATE, CHANNEL, TEXT, SENTIMENT, EMOTION, INFO, URI]

def looks_like_record(tokens):
    """Heuristic: last-2 must be allowed INFO; col 2 is ISO-Z date; plausible ID length."""
    if len(tokens) < 8:
        return False
    info = tokens[-2].strip()
    if info not in INFO_SET:
        return False
    if ISO_Z_PATTERN.match(tokens[1].strip()) is None:
        return False
    if len(tokens[0].strip()) < 16:
        return False
    return True

def csv_repair(in_path, out_path, encoding="utf-8"):
    fixed = total = bad = 0
    header_written = False

    with open(in_path, "r", encoding=encoding, errors="replace") as fin, \
         open(out_path, "w", encoding="utf-8", newline="") as fout:

        writer = csv.writer(fout, delimiter=";", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        buffer = ""
        header = None

        for raw in fin:
            line = raw.rstrip("\n")
            if not line.strip() and not buffer:
                continue

            if not header_written:
                header = line
                writer.writerow(["ID","DATE","CHANNEL","TEXT","SENTIMENT","EMOTION","INFO","URI"])
                header_written = True
                continue

            buffer = (buffer + "\n" + line) if buffer else line
            tokens = [t for t in buffer.split(";")]

            if not looks_like_record(tokens):
                continue

            total += 1
            try:
                _id       = tokens[0].strip()
                _date     = tokens[1].strip()
                _channel  = tokens[2].strip()
                _sent     = tokens[-4].strip()
                _emo      = tokens[-3].strip()
                _info     = tokens[-2].strip()
                _uri      = ";".join(tokens[-1:]).strip()
                _text     = ";".join(tokens[3:-4]).strip().replace('"', '""')

                writer.writerow([_id, _date, _channel, _text, _sent, _emo, _info, _uri])
                if len(tokens) > 8:
                    fixed += 1

            except Exception:
                bad += 1
            finally:
                buffer = ""

        if buffer.strip():
            bad += 1

    return dict(total_records=total, repaired_with_extra_semicolons=fixed, irreparable=bad)

stats = csv_repair(INPUT_CSV, REPAIRED_CSV)
print("Repair completed:", stats)
print("Repaired file written to:", REPAIRED_CSV)

Repair completed: {'total_records': 218506, 'repaired_with_extra_semicolons': 21889, 'irreparable': 0}
Repaired file written to: /content/dataset_gbv_repaired.csv


## Load, clean allowed values, and apply filters

In [6]:
df = pd.read_csv(REPAIRED_CSV, sep=";", dtype=str, keep_default_na=False, na_values=[])

for c in ["SENTIMENT","EMOTION","INFO","CHANNEL"]:
    if c in df:
        df[c] = df[c].astype(str).str.strip().str.upper()

mask_sent_bad = ~df["SENTIMENT"].isin(ALLOWED_SENTIMENT)
df.loc[mask_sent_bad, "SENTIMENT"] = "-"

mask_emo_bad = ~df["EMOTION"].isin(ALLOWED_EMOTIONS)
df.loc[mask_emo_bad, "EMOTION"] = pd.NA

mask_keep_info = (df["INFO"] == "INFO")

before = len(df)
df = df[mask_keep_info].copy()
df = df[df["EMOTION"].notna()].copy()
after = len(df)
print(f"Rows before: {before} | after filters: {after} | removed: {before - after}")


Rows before: 218506 | after filters: 71609 | removed: 146897


## Sanity checks + final save

In [7]:
expected_cols = ["ID","DATE","CHANNEL","TEXT","SENTIMENT","EMOTION","INFO","URI"]
assert list(df.columns) == expected_cols, f"Unexpected columns: {df.columns.tolist()}"
assert df["DATE"].str.match(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$").all(), "DATE not ISO-Z on some rows"
assert df["INFO"].eq("INFO").all(), "INFO filter failed"
assert df["SENTIMENT"].isin(ALLOWED_SENTIMENT).all(), "Invalid SENTIMENT values"
assert df["EMOTION"].isin(ALLOWED_EMOTIONS).all(), "Invalid EMOTION values"

print("Final shape (rows, cols):", df.shape)
df.to_csv(CLEAN_CSV, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)
print("Clean file written to:", CLEAN_CSV)


Final shape (rows, cols): (71609, 8)
Clean file written to: /content/dataset_gbv_clean.csv


## Analysis of clean file

In [8]:
CLEAN_CSV   = "/content/dataset_gbv_clean.csv"
OUT_REPORT  = "/content/gbv_counts_summary.csv"
ALLOWED_INFO_LIST      = ["INFO", "NOINFO", "-"]
ALLOWED_SENTIMENT_LIST = ["POS", "NEG", "-"]
ALLOWED_EMOTIONS_LIST  = ["GIOIA","TRISTEZZA","RABBIA","PAURA","DISGUSTO","SORPRESA","FIDUCIA","ATTESA","NEUTRA"]

def analyze_clean_file(path=CLEAN_CSV, out_report=OUT_REPORT):
    df = pd.read_csv(path, sep=";", dtype=str, keep_default_na=False, na_values=[])

    expected = ["ID","DATE","CHANNEL","TEXT","SENTIMENT","EMOTION","INFO","URI"]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in clean CSV: {missing}")

    for c in ["INFO","SENTIMENT","EMOTION"]:
        df[c] = df[c].astype(str).str.strip().str.upper()

    total_rows = len(df)
    info_counts = df["INFO"].value_counts().reindex(ALLOWED_INFO_LIST, fill_value=0)
    sent_counts = df["SENTIMENT"].value_counts().reindex(ALLOWED_SENTIMENT_LIST, fill_value=0)
    emo_counts  = df["EMOTION"].value_counts().reindex(ALLOWED_EMOTIONS_LIST, fill_value=0)

    print(f"Total rows: {total_rows}\n")
    print("Counts by INFO (INFO/NOINFO/-):")
    print(info_counts.to_string(), "\n")
    print("Counts by SENTIMENT (POS/NEG/-):")
    print(sent_counts.to_string(), "\n")
    print("Counts by EMOTION:")
    print(emo_counts.to_string(), "\n")

    report_rows = []
    for label, cnt in info_counts.items():
        report_rows.append({"group":"INFO", "label":label, "count":int(cnt)})
    for label, cnt in sent_counts.items():
        report_rows.append({"group":"SENTIMENT", "label":label, "count":int(cnt)})
    for label, cnt in emo_counts.items():
        report_rows.append({"group":"EMOTION", "label":label, "count":int(cnt)})

    report_df = pd.DataFrame(report_rows, columns=["group","label","count"])
    report_df.to_csv(out_report, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"Report saved to: {out_report}")

    return {
        "total_rows": total_rows,
        "info_counts": info_counts,
        "sent_counts": sent_counts,
        "emo_counts": emo_counts,
        "report_df": report_df,
    }

_ = analyze_clean_file()

Total rows: 71609

Counts by INFO (INFO/NOINFO/-):
INFO
INFO      71609
NOINFO        0
-             0 

Counts by SENTIMENT (POS/NEG/-):
SENTIMENT
POS    37424
NEG    13479
-      20706 

Counts by EMOTION:
EMOTION
GIOIA        14967
TRISTEZZA    12442
RABBIA       14027
PAURA          568
DISGUSTO         0
SORPRESA      1394
FIDUCIA          0
ATTESA           0
NEUTRA       28211 

Report saved to: /content/gbv_counts_summary.csv


## Deduplicate TEXT + post-dedup analysis

In [11]:
# =========================================================
# Remove Twitter prefixes from TEXT
# Pattern: <something>@twitter.com (User Name) [RT ]@<handle>...
# Output: cleaned dataset + audit with original and cleaned text
# =========================================================
IN_CSV    = "/content/dataset_gbv_clean.csv"
OUT_CSV   = "/content/dataset_gbv_clean2.csv"
AUDIT_CSV = "/content/gbv_twclean_audit.csv"

def strip_twitter_prefix(text: str):
    """
    Remove leading
      <something>@twitter.com (User Name) [RT ]@<handle>...
    Returns (new_text, was_modified)
    """
    if not isinstance(text, str):
        return text, False

    pattern = re.compile(
        r"^.+?@twitter\.com\s*\([^)]*\)\s*(?:RT\s+)?(?:@\w+\s*)+",
        re.IGNORECASE
    )
    new_text, n = pattern.subn("", text, count=1)
    return new_text.strip(), n > 0

# 1) Load
df = pd.read_csv(IN_CSV, sep=";", dtype=str, keep_default_na=False, na_values=[])

# 2) Strip prefixes
mask = df["CHANNEL"].astype(str).str.upper().eq("TWITTER")
subset = df.loc[mask].copy()
subset["TEXT_ORIG"] = subset["TEXT"]
subset["TEXT_STRIPPED"], subset["_TW_PREFIX_REMOVED"] = zip(*subset["TEXT"].map(strip_twitter_prefix))
df.loc[subset.index, "TEXT"] = subset["TEXT_STRIPPED"]

# 3) Audit
audit = subset.loc[subset["_TW_PREFIX_REMOVED"], ["ID","DATE","CHANNEL","TEXT_ORIG","TEXT_STRIPPED"]].copy()
audit.rename(columns={"TEXT_STRIPPED": "TEXT_NEW"}, inplace=True)

# 4) Save
df.to_csv(OUT_CSV, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)
audit.to_csv(AUDIT_CSV, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)

print(f"Cleaning completed. Modified rows: {len(audit)}")
print(f"Cleaned dataset: {OUT_CSV}")
print(f"Audit file:      {AUDIT_CSV}")


Cleaning completed. Modified rows: 7444
Cleaned dataset: /content/dataset_gbv_clean2.csv
Audit file:      /content/gbv_twclean_audit.csv


In [12]:
# =========================================================
# Dedup "truncated tweets": texts ending with "..." or "…"
# Consider duplicates if they are prefixes of longer versions.
# Keep the longest version. Count duplicates overall, by CHANNEL, by EMOTION.
# =========================================================
IN_CSV  = "/content/dataset_gbv_clean2.csv"
OUT_CSV = "/content/dataset_gbv_clean3.csv"
DUP_CSV = "/content/gbv_twclean_ellipsis_duplicates.csv"

ELLIPSIS_END_RE = re.compile(r"(?:\s*(?:\.{3}|…))+$")

def normalize_text(s: str) -> str:
    """Lightweight normalization to compare near-identical texts."""
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")
    s = s.casefold()
    s = re.sub(r"\s+", " ", s).strip()
    return s

def strip_trailing_ellipsis_norm(tnorm: str) -> Tuple[str, bool]:
    """Remove trailing ellipsis from normalized text; returns (prefix_norm, was_truncated?)."""
    new = ELLIPSIS_END_RE.sub("", tnorm)
    return new, (new != tnorm)

# 1) Load
df = pd.read_csv(IN_CSV, sep=";", dtype=str, keep_default_na=False, na_values=[])
df["CHANNEL"] = df["CHANNEL"].astype(str).str.strip().str.upper()
df["EMOTION"] = df["EMOTION"].astype(str).str.strip().str.upper()

# 2) Features for dedup
df["_TNORM"] = df["TEXT"].map(normalize_text)
df["_LEN"]   = df["_TNORM"].str.len()
df["_PREFIX"], df["_IS_TRUNC"] = zip(*df["_TNORM"].map(strip_trailing_ellipsis_norm))

# 3) Build group key
df["_GROUP_KEY"] = df["_TNORM"]
tnorm_series = df["_TNORM"]
length_series = df["_LEN"]
trunc_idx = df.index[df["_IS_TRUNC"]].tolist()

for i in trunc_idx:
    pref = df.at[i, "_PREFIX"]
    if not pref:
        continue
    mask = tnorm_series.str.startswith(pref)
    if not mask.any():
        continue
    cand = df.loc[mask, ["_TNORM","_LEN"]].copy()
    max_len = cand["_LEN"].max()
    best_tnorm = cand.loc[cand["_LEN"] == max_len, "_TNORM"].iloc[0]
    df.at[i, "_GROUP_KEY"] = best_tnorm

# 4) Representative per group
df["_ROW_ID"] = range(len(df))
rep_df = (
    df.sort_values(["_GROUP_KEY","_LEN","_ROW_ID"], ascending=[True, False, True])
      .groupby("_GROUP_KEY", as_index=False)
      .head(1)[["_GROUP_KEY","_ROW_ID"]]
      .rename(columns={"_ROW_ID":"_REP_ROW_ID"})
)
df = df.merge(rep_df, on="_GROUP_KEY", how="left")
duplicate_mask = df["_ROW_ID"] != df["_REP_ROW_ID"]

df["_DUP_REASON"] = pd.Series(["exact"] * len(df))
df.loc[df["_GROUP_KEY"] != df["_TNORM"], "_DUP_REASON"] = "ellipsis"

# 5) Counts
total_removed = int(duplicate_mask.sum())
by_channel = df.loc[duplicate_mask, "CHANNEL"].value_counts().sort_index()
by_emotion = df.loc[duplicate_mask, "EMOTION"].value_counts().sort_index()

print("=== DUPLICATES TO REMOVE (ELLIPSIS) ===")
print(f"Total: {total_removed}\n")
print("By CHANNEL:")
print(by_channel.to_string() if not by_channel.empty else "(none)", "\n")
print("By EMOTION:")
print(by_emotion.to_string() if not by_emotion.empty else "(none)")

# 6) Save audit and final dataset
dups = df.loc[duplicate_mask, ["ID","DATE","CHANNEL","EMOTION","TEXT","_DUP_REASON","_GROUP_KEY"]].copy()
dups.rename(columns={"TEXT":"TEXT_REMOVED", "_GROUP_KEY":"GROUP_KEY_TNORM"}, inplace=True)

rep_map = df.set_index("_ROW_ID")[["TEXT"]].to_dict()["TEXT"]
df["_REP_TEXT"] = df["_REP_ROW_ID"].map(rep_map)
dups["TEXT_KEPT"] = df.loc[dups.index, "_REP_TEXT"]

df_nodup = df.loc[~duplicate_mask, [c for c in df.columns if not c.startswith("_")]]
df_nodup = df_nodup.drop(columns=["_REP_ROW_ID","_ROW_ID"], errors="ignore")

df_nodup.to_csv(OUT_CSV, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)
dups.to_csv(DUP_CSV, sep=";", index=False, quoting=csv.QUOTE_MINIMAL)

print("Dedup (ellipsis) completed.")
print("Dataset without duplicates:", OUT_CSV)
print("Duplicates audit (removed vs kept):", DUP_CSV)


=== DUPLICATES TO REMOVE (ELLIPSIS) ===
Total: 1506

By CHANNEL:
CHANNEL
FACEBOOK     1111
FEED            2
INSTAGRAM     145
TWITTER       248 

By EMOTION:
EMOTION
GIOIA        194
NEUTRA       592
PAURA          9
RABBIA       343
SORPRESA      36
TRISTEZZA    332
Dedup (ellipsis) completed.
Dataset without duplicates: /content/dataset_gbv_clean3.csv
Duplicates audit (removed vs kept): /content/gbv_twclean_ellipsis_duplicates.csv
